## baseline

In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 127.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=a49d279cdce808d650337f9989351d20c06ff199799439e512e284986c6d58ce
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 49.9 MB/s eta 0:00:00


In [ ]:
# Mecab이 제일 빠르긴 한데 로컬에서 설치하기 빡세니까 Okt 사용
from konlpy.tag import Okt
#from konlpy.tag import Mecab

okt = Okt()
okt.nouns('우리과 사무실 전화번호좀 알려줘.')

['우리', '사무실', '전화번호', '좀']

In [ ]:
# 전처리 확인 코드 -> 사용XXXXXXXXX
import re

def get_clean_text(df):
    text = []
    
    for i in range(0, len(df)):
        if (str(df) =='nan') : # 지우고 싶은 글자가 있는 컬럼 
            temp = ''
        else : 
            temp = str(df)
            temp = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…《\》]', '', temp) # 특수문자
            temp = re.sub('([ㄱ-ㅎㅏ-ㅣ]+)', '', temp) # 한글 자음, 한글 모음
            temp = re.sub('([♡❤✌❣♥ᆢ✊❤️✨⤵️☺️;”“]+)', '', temp) # 이모티콘 
            only_BMP_pattern = re.compile("["
                                u"\U00010000-\U0010FFFF"  #BMP characters 이외
                               "]+", flags=re.UNICODE)
            temp = only_BMP_pattern.sub(r'', temp)# BMP characters만
            emoji_pattern = re.compile("["
                                    u"\U0001F600-\U0001F64F"  # emoticons
                                    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                    u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                                       "]+", flags=re.UNICODE)
            temp=  emoji_pattern.sub(r'', temp) # 유니코드로 이모티콘 지우기
            text.append(temp)
            
        text1 = " ".join(text)
            
    return text1

In [ ]:
import torch
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# 확인용 모델 import -> 전에 import 해둔 모델하고 같은모델
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

## file load

In [ ]:
# csv load
df_department = pd.read_csv('department_data.csv', encoding = 'utf-8') # 학과정보
df_sju = pd.read_csv('sju_data.csv', encoding = 'utf-8') # 세종대 정보
df_curri = pd.read_csv('curriculum_16_23.csv', encoding = 'utf-8') # 학번 별 커리큘럼

FileNotFoundError: ignored

In [ ]:
# 모델 embedding data load
em_department = torch.load('department_data.pt')
em_sju = torch.load('sju_data.pt')
em_curri = torch.load('curriculum_16_23.pt')

In [ ]:
# sentence_transformers local 사용 제한으로 인한 유사도 선언
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
        return dot(A, B)/(norm(A)*norm(B))

intent에 있는 단어 -> 단어사전에 활용\
단어사전은 사용자의 질문이 학교관련 질문인가에 대한 판단 지표로 활용

In [ ]:
depart_word_dic = df_department['intent'].values.tolist()
depart_word_dic = list(set(depart_word_dic))
depart_word_dic

In [ ]:
query_word_dic = df_sju['intent'].values.tolist()
query_word_dic = list(set(query_word_dic))
query_word_dic

In [ ]:
curri_word_dic = df_curri['intent'].values.tolist()
curri_word_dic = list(set(curri_word_dic))
curri_word_dic

## UNIV word dictionary 구축

In [ ]:
## 단어사전 구축
## 단어사전 -> 사전 내에 질문지 해당내용 없을경우 -> 일상대화 모델
import urllib.request
import pandas as pd
from konlpy.tag import Mecab
from nltk import FreqDist
import numpy as np
import matplotlib.pyplot as plt

# Univ_dict = query_word_dic + depart_word_dic + curri_word_dic
# 정답지에 있는 문장들 다 끌어와서 단어사전 구축
depart_ans_dic = df_department['answer'].values.tolist()
query_ans_dic = df_sju['answer'].values.tolist()
curri_ans_dic = df_curri['answer'].values.tolist()

Univ_dict = depart_ans_dic + query_ans_dic + curri_ans_dic
for i in range(len(Univ_dict)):
    Univ_dict[i] = str(Univ_dict[i]).replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

# 불용어 정의
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

tokenizer = Okt()
tokenized=[]

# 문장 토큰화 및 전처리
for sentence in Univ_dict:
    temp = tokenizer.nouns(sentence) # 토큰화
    temp = [word for word in temp if not word in stopwords] # 불용어 제거
    tokenized.append(temp)

vocab = FreqDist(np.hstack(tokenized))

print('단어 집합의 크기 : {}'.format(len(vocab)))

dict(vocab)



In [ ]:
# 한글자인 단어 제거
filtered_vocab = {key: value for key, value in vocab.items() if len(key) > 1}
sorted_dictionary = dict(sorted(filtered_vocab.items(), key=lambda x: x[1], reverse=True))
top_500_items = dict(list(sorted_dictionary.items())[:500])

print('단어 집합의 크기 : {}'.format(len(top_500_items)))


In [ ]:
top_500_items

In [ ]:
# 단어사전 구축

Univ_dict_final = query_word_dic + depart_word_dic + curri_word_dic
Univ_dict_final += list(top_500_items)
len(Univ_dict_final)

## test

In [ ]:
# user 정보
# 정보 얻는 방식은 sju data + 자기 해당 학과 정보만 저장해서 임베딩 하는 방식.

# 학과 학번
user_id = 18
user_depart = '컴퓨터공학과'

# 해당학과 질문지 추출
user_DQ = df_department[df_department['intent'] == user_depart]
# 해당학과 질문지 index 값 추출
user_DQ_idx = df_department.loc[df_department['intent'] == user_depart].index
user_depart_embed = em_department[user_DQ_idx]

# 해당 학번 커리큘럼 추출
user_curri = df_curri[df_curri['intent']==user_id]
# 해당 학번 커리큘럼 질문지 index 값 추출
user_curri_idx = df_curri.loc[df_curri['intent'] == user_id].index
user_curri_embed = em_curri[user_curri_idx]


In [ ]:
# 세 데이터프레임 합치기
concat_sju = pd.concat([df_sju,user_DQ,user_curri])

print('sju_data length',len(df_sju),'user department length',len(user_DQ),'user id curri',len(user_curri))
print('concat sju length',len(concat_sju))

# 임베딩값 합치기 -> tensor 형태에서는 차원이 달라 합칠수 없음
# 불가피하게 리스트로 변경 후 합침
em_sju_list = em_sju.tolist()
user_depart_embed_list = user_depart_embed.tolist()
user_curri_embed_list = user_curri_embed.tolist()
concat_sju_embed = em_sju_list + user_depart_embed_list + user_curri_embed_list
print('sju_data embed length',len(em_sju_list),'user department embed length',len(user_depart_embed_list),'user curri embed list',len(user_curri_embed_list))
print('concat sju embed length',len(concat_sju_embed))

In [ ]:
# 해당 사용자만의 질문,응답 리스트 생성
sju_answer = concat_sju['answer'].values.tolist()
sju_query = concat_sju['query'].values.tolist()

In [ ]:
# 토크나이저 사용을 위한 토큰 선언
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [ ]:
from google.colab import files
files.upload('/content/drive/MyDrive/epoch8model.pt')

In [ ]:
# 일상대화 모델 답변 출력 부분 
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

# 모델 선언, 토크나이저 선언 부분은 따로 해둬야할듯?
device = "cuda" if torch.cuda.is_available() else "cpu"
our_model = torch.load('epoch5model.pt')
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 


# 답변 생성 과정 -> user_text에 대한 답변
def conv_model(text):
     #with torch.no_grad():
        q = text
        a = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0).to(device)
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().cpu().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        conv_ans = a.strip()
        return str(conv_ans)

In [ ]:
user_text = ''

In [ ]:
# user 질문 임베딩값, 텐서
user_encode = model.encode(user_text)
user_tensor = torch.tensor(user_encode)

cos_similarity = []
count = 0

# 질문지 유사도 탐색
for i in range(len(concat_sju_embed)):
        cos_similarity.append(cos_sim(user_tensor, concat_sju_embed[i]))

# 유사도 가장 높은 답변 탐색 및 출력
best_sim_idx = int(np.argmax(cos_similarity))
cos_sim_value = np.max(cos_similarity)

# 유사도 0.5보다 낮은경우는 우리 질문지에 없는 질문.
if cos_sim_value <= 0.5:
    answer = '준비되지 않은 답변입니다!!'
    user_text_nouns =  tokenizer.nouns(user_text)
    for noun in user_text_nouns:
        if noun in Univ_dict_final:
            print('단어사전 내에 정보가 있음')
            count += 1
            break

    # 단어사전내에 정보가 있음 -> 일상대화 x / 준비중인 답변
    if count > 0 :
        answer = '준비되지 않은 답변입니다!'

    else:
    # 일상대화 출력
        answer = conv_model(user_text)

    # 이부분에서 잘못된 정답일 경우 -> 일상대화 모델?
    # 단어사전 탐색 후  있으면 답변이 준비중 // 아니면 일상대화
else : 
    answer = sju_answer[best_sim_idx]

print(cos_sim_value)
print(sju_query[best_sim_idx])
print(answer)